# Visualising with LavaVu

[LavaVu](https://github.com/OKaluza/LavaVu) is a lightweight, automatable visualisation and analysis viewing utility.

> This notebook assumes you have run `distributed_mesh.py` in the tests folder with one or more processors.

How to load HDF5 data with h5py and display with LavaVu...

In [ ]:
import lavavu
import h5py
import numpy

f = h5py.File('../tests/spiral.h5', 'r')

#Print some info about the data structures
lavavu.printH5(f)

for field in f['fields']:
    print(field)
for label in f['labels']:
    print(label)

In [ ]:
topography = f['fields']['height']
vertices = f['geometry']['vertices']
cells = f['viz']['topology']['cells']

boundary = f['labels']['boundary']['1']['indices']

In [ ]:
#print topography[0:10],topography.shape
#print vertices[0:10],vertices.shape
#print cells[0:10],cells.shape
#print boundary[0:10],topography.shape

**Create a viewer**

In [ ]:
lv = lavavu.Viewer(border=False, background="darkgrey")

**Plot a triangle surface**

Can be vertices only (3 per tri) or vertices (shared) with indices (3 per tri)

In [ ]:
#Convert the vertex & vector arrays to 3d with numpy
verts = numpy.reshape(vertices, (-1,2))
#Insert the topography layer in Y axis
verts = numpy.insert(verts, 2, values=topography, axis=1)

In [ ]:
#Plot the triangles
tris = lv.triangles("surface")
tris.vertices(verts)
tris.indices(cells)

**Add values, can be used to colour and filter the data**

In [ ]:
#Use topography value to colour the surface
tris.values(topography, 'topography')
#tris.values(boundary, "boundary") #Load another field
#tris.values(tin["discharge"])
cm = tris.colourmap("diverge") #Apply a built in colourmap
cb = tris.colourbar() #Add a colour bar

#Filter by min height value
tris["zmin"] = 0.011

**Apply an initial rotation and display an interactive viewer window**

Along with viewer window, controls can be added to adjust properties dynamically

In [ ]:
lv.rotate('x', -60)

lv.window()
tris.control.Checkbox('wireframe')
tris.control.Range('zmin', range=(0,1), step=0.001)
lv.control.Range(command='background', range=(0,1), step=0.1, value=1)
lv.control.ObjectList()
lv.control.show()

In [ ]:
tris.reload()
lv.redisplay()

**Saving camera position, images and state**



In [ ]:
lv.camera() #Get current camera as set in viewer

In [ ]:
#Plot a static image with a saved camera setup
lv.translation(0.00558684254065156, 1.16282057762146, -8.33703231811523)
lv.rotation(-0.487509340047836, 0.172793388366699, 0.269237726926804, 0.807206630706787)
lv.display(resolution=[480,320])
lv.image('saved.png', resolution=[640,480], transparent=True) #Save to disk

In [ ]:
#State of properties and view can be loaded and saved to a file
lv.save("state.json")
#lv.file("state.json")
